# Policy search
Policy search RL algorithm for playing the game of Space Invaders.

In [58]:
import random

import ale_py
import gymnasium as gym

gym.register_envs(ale_py)  # needed to run atari games

In [59]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## preprocess
We need to crop the frame, downsample it, and convert it to grayscale.

In [60]:
def preprocess_frame(frame):
    frame = frame[34:194]
    frame = frame[::2, ::2, 0]
    frame[frame == 144] = 0
    frame[frame == 109] = 0
    frame[frame != 0] = 1
    return np.expand_dims(frame.astype(np.float32), axis=0)

In [61]:
class PolicyNetwork(nn.Module):
    def __init__(self, action_space):
        super(PolicyNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=4, stride=2),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, action_space)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return nn.Softmax(dim=-1)(x)

In [62]:
def compute_discounted_rewards(rewards, gamma, baseline=0):
    discounted_rewards = np.zeros_like(rewards)
    cumulative = 0
    for t in reversed(range(len(rewards))):
        cumulative = rewards[t] + gamma * cumulative
        discounted_rewards[t] = cumulative
    return discounted_rewards - baseline

In [ ]:
def create_env(seed):
    def _init():
        env = gym.make('ALE/SpaceInvaders-v5', frameskip=3)
        env.reset(seed=seed)
        return env
    return _init

In [63]:
def play_one_step(env, state, model):
    # get stochastic action
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    action_probs = model(state)
    dist = Categorical(action_probs)
    action = dist.sample()
    log_prob = dist.log_prob(action)

    # take action
    next_state, reward, done, truncated, info = env.step(action.item())
    next_state = preprocess_frame(next_state)
    return next_state, reward, done, truncated, log_prob


def play_multiple_episodes(env, n_episodes, n_max_steps, model, seed, gamma):
    all_rewards = []
    all_log_probs = []
    all_baselines = []

    for i in range(n_episodes):
        current_rewards = []
        current_log_probs = []
        adjusted_seed = seed + i
        state, _ = env.reset(seed=adjusted_seed)
        state = preprocess_frame(state)

        for i in range(n_max_steps):
            state, reward, done, truncated, log_prob = play_one_step(env, state, model)
            current_rewards.append(reward)
            current_log_probs.append(log_prob)
            if done or truncated:
                break

        discounted = compute_discounted_rewards(current_rewards, gamma)
        all_rewards.append(current_rewards)
        all_log_probs.append(current_log_probs)
        all_baselines.append(np.mean(discounted))

    return all_rewards, all_log_probs, all_baselines

In [64]:
class Parameters:
    def __init__(self):
        self.n_iterations = 1000  # max episodes
        self.n_episodes_per_update = 10  # batch size
        self.n_max_steps = 1_000_000_000  # how many steps to perform during a game

        self.discount_factor = 0.99  # gamma
        self.learning_rate = 1e-2  # alfa

        self.early_stopping_patience = 20
        self.early_stopping_delta = 1e-3

In [65]:
env = gym.make('ALE/SpaceInvaders-v5', frameskip=3)
# env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=50) ??? nie dziala dziadostwo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [66]:
n_iterations = 1000
n_episodes_per_update = 10
n_max_steps = 1_000_000_000
discount_factor = 0.99  # gamma
learning_rate = 1e-2  # alfa

In [67]:
seed_rewards = []
early_stopping_patience = 20
early_stopping_delta = 1e-3

# Train the model for each seed
for seed in [1, 2, 3, 5, 8]:
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    episode_rewards = []
    best_mean_reward = -np.inf
    patience_counter = 0

    policy = PolicyNetwork(env.action_space.n).to(device)
    optimizer = optim.Adam(policy.parameters(), lr=learning_rate)

    for iteration in range(n_iterations):
        all_rewards, all_log_probs, all_baselines = play_multiple_episodes(
            env, n_episodes_per_update, n_max_steps, policy, seed, gamma=discount_factor
        )

        all_final_rewards = [
            compute_discounted_rewards(rewards, discount_factor, baseline=baseline)
            for rewards, baseline in zip(all_rewards, all_baselines)
        ]

        policy_loss = 0
        for log_probs, rewards in zip(all_log_probs, all_final_rewards):
            log_probs_tensor = torch.stack(log_probs)
            rewards_tensor = torch.tensor(rewards, dtype=torch.float32, device=device)
            policy_loss += -torch.sum(log_probs_tensor * rewards_tensor)

        policy_loss /= n_episodes_per_update

        episode_rewards.append(np.mean([np.sum(rewards) for rewards in all_rewards]))
        if iteration % 10 == 0:
            print(f"Seed: {seed}, Iteration: {iteration}, Reward: {episode_rewards[-1]}")

        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        # Early stopping
        if iteration < 50:
            continue
        current_mean_reward = np.mean(episode_rewards[-50:])
        if abs(current_mean_reward - best_mean_reward) > early_stopping_delta:
            best_mean_reward = current_mean_reward
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter == early_stopping_patience:
            print(f"Seed: {seed}, Early stopping at iteration {iteration}")
            break

    seed_rewards.append(episode_rewards)

env.close()

Seed: 1, Iteration: 0, Reward: 184.5
Seed: 1, Iteration: 10, Reward: 141.0
Seed: 1, Iteration: 20, Reward: 120.0
Seed: 1, Iteration: 30, Reward: 102.5


KeyboardInterrupt: 

In [ ]:
rewards_for_plot = [[reward for reward in rewards] for rewards in seed_rewards]
rewards_for_plot = pd.DataFrame(rewards_for_plot).melt()
rewards_for_plot.rename(columns={"variable": "episode", "value": "reward"}, inplace=True)
sns.set(style="whitegrid", palette="muted")
sns.lineplot(x="episode", y="reward", data=rewards_for_plot)
plt.show()